In [1]:
import pandas as pd

In [2]:
patch_responses = pd.read_csv('resources/linux/resources/filtered_responses.csv')

In [3]:
patch_responses.head()

,Unnamed: 0,patch_id,response_author,resp_parent,resp_msg_id,upstream,response_is_bot,bot_name,patch_author,responder,self_response
0,0,<0-v1-0bc346e08476+585-drop_offsetofend_jgg@me...,Jason Gunthorpe <jgg@ziepe.ca>,<0-v1-0bc346e08476+585-drop_offsetofend_jgg@me...,<20200529182915.GA18558@ziepe.ca>,NaN,False,NaN,"('jason gunthorpe', 'jgg@ziepe.ca')","('jason gunthorpe', 'jgg@ziepe.ca')",True
1,1,<0-v1-0bc346e08476+585-drop_offsetofend_jgg@me...,Jason Gunthorpe <jgg@ziepe.ca>,<0-v1-0bc346e08476+585-drop_offsetofend_jgg@me...,<20200529182915.GA18558@ziepe.ca>,ffd7339a2fac98b9ff731e336c4411bf1ce57e22,False,NaN,"('jason gunthorpe', 'jgg@ziepe.ca')","('jason gunthorpe', 'jgg@ziepe.ca')",True
2,2,<0-v1-4eb72686de3c+5062-hmm_no_flags_jgg@mella...,Jason Gunthorpe <jgg@ziepe.ca>,<0-v1-4eb72686de3c+5062-hmm_no_flags_jgg@mella...,<2-v1-4eb72686de3c+5062-hmm_no_flags_jgg@mella...,NaN,False,NaN,"('jason gunthorpe', 'jgg@ziepe.ca')","('jason gunthorpe', 'jgg@ziepe.ca')",True
3,3,<0-v1-4eb72686de3c+5062-hmm_no_flags_jgg@mella...,Ralph Campbell <rcampbell@nvidia.com>,<0-v1-4eb72686de3c+5062-hmm_no_flags_jgg@mella...,<17ce2fdc-4f9f-7772-c10c-6f339a4183e8@nvidia.com>,NaN,False,NaN,"('jason gunthorpe', 'jgg@ziepe.ca')","('ralph campbell', 'rcampbell@nvidia.com')",False
4,4,<0-v1-4eb72686de3c+5062-hmm_no_flags_jgg@mella...,Jason Gunthorpe <jgg@ziepe.ca>,<0-v1-4eb72686de3c+5062-hmm_no_flags_jgg@mella...,<3-v1-4eb72686de3c+5062-hmm_no_flags_jgg@mella...,NaN,False,NaN,"('jason gunthorpe', 'jgg@ziepe.ca')","('jason gunthorpe', 'jgg@ziepe.ca')",True


In [4]:
response_counts = patch_responses.groupby('upstream').nunique('resp_msg_id')[['resp_msg_id']].reset_index()

In [5]:
response_counts[response_counts['resp_msg_id'] == 0][['upstream']].to_csv('no_response.csv', index=False)

In [6]:
from pypasta import Config
from pypasta.Repository.Mbox import Mbox, PatchMail

config = Config('linux')
mailbox = Mbox(config)

def get_email(message_id):
    try:
        return mailbox.get_messages(message_id)
    except:
        raise

def get_tags(email, tag_name):
    try:
        return PatchMail(email, email['Message-Id']).tags[tag_name]
    except:
        return None
    
def get_header(email, header):
    try:
        return email[header]
    except:
        return None

ValueError: time data '4Y/08/22' does not match format '%Y/%m/%d'

In [ ]:
patch_responses['email'] = patch_responses['resp_msg_id'].map(get_email)

In [ ]:
def try_reduce(item):
    try:
        return item[0]
    except:
        return None

patch_responses['author'] = patch_responses['email']\
                                .apply(lambda email: try_reduce(get_tags(email, 'signed-off-by')))

Signed-off-by is the most consistently used tag considering each mail. This is used to determine the original author of patches in emails. Only the first Signed-off-by tag is used since these tags increase as patches evolve.

In [ ]:
patch_responses['reviewed-by'] = patch_responses['email']\
                                .map(lambda email: get_tags(email, 'reviewed-by'))

In [ ]:
cols

In [ ]:
patch_responses.drop('email', axis=1, inplace=True)

In [ ]:
cols = list(patch_responses.columns)
cols.remove('reviewed-by')
patch_responses_long = patch_responses.set_index(cols)

In [ ]:
patch_responses_long = pd.melt(patch_responses_long['reviewed-by'].apply(pd.Series).reset_index(), 
            id_vars=cols,
            value_name='reviewed-by').sort_index()
patch_responses_long.drop('variable', axis=1, inplace=True)

Number of unique developers and reviewers that are in tha dataset:

In [ ]:
patch_responses_long[['author', 'reviewed-by']].nunique()

Total number of developer and reviewer emails that are in tha dataset:

In [ ]:
patch_responses_long[['author', 'reviewed-by']].count()

In [ ]:
patch_responses_long.to_csv('df_with_tags.csv')

A very small portion of the response emails are coming from reviewers.

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt

##### Number of email responses of each Developer for all patches included in some fixed release, e.g., v5.5


In [ ]:
response_counts = patch_responses[['author','resp_msg_id']]\
                    .groupby('author')\
                    .nunique()['resp_msg_id']\
                    .reset_index()\
                    .sort_values('resp_msg_id', ascending=False)
# response_counts.head()

The developers who receieved the most responses

In [ ]:
ax = response_counts[:10].plot(x = 'author', title = 'Authors with the most responses', kind = 'barh')
ax.figure.savefig('authors_most_responses.png', dpi=1000, bbox_inches = "tight")

In [ ]:
del response_counts

In [ ]:
ax = patch_responses[['author','upstream']].groupby('author')\
                .nunique()['upstream']\
                .reset_index('author')\
                .sort_values('upstream', ascending=False)\
                .head(10)\
                .plot(title = 'Authors who developed the largest number of patches', kind = 'barh', x = 'author')
ax.figure.savefig('authors_most_commits.png', dpi=1000, bbox_inches = "tight")

Above are the top 20 authors who has been associated with the largest number of distinct upstream commits. Authors are extracted from emails. The first signed-off-by tag in each email is assumed to be the author/developer of the patch discussed in that email.

#### Number of Reviewed-by of each Developer (in some fixed release)

In [ ]:
review_counts = patch_responses_long[['author', 'reviewed-by', 'resp_msg_id']]\
                    .groupby('author')\
                    .count()['reviewed-by']\
                    .reset_index()\
                    .sort_values('reviewed-by', ascending=False)
review_counts.head()

In [ ]:
ax = review_counts.plot(x = 'author', title = 'Distribution of review counts per author', kind = 'hist', bins = 20)
ax.figure.savefig('authors_review_distribution.png', dpi=1000, bbox_inches = "tight")

Above is the distribution of the amout of review emails each developer receive. Data is divided into 20 bins. Nevertheless, it can be seen that a large percentage of the developers received a small amount of reviews.

In [ ]:
ax.figure.savefig('authors_most_reviews.png', dpi=1000, bbox_inches = "tight")

Above are the top 10 authors who has received the largest number of review emails. For these counts, every email which has a Reviewed-by tag is considered a review.

In [ ]:
del review_counts

#### Compare number of Reviewed-by vs. email responses per patch
(Is there a correlation on patch basis? E.g., more email responses, more Reviewed-by tags?)

In [ ]:
patches_emails = patch_responses_long\
                    .groupby('upstream')\
                    .nunique()['resp_msg_id']
patches_reviews = patch_responses_long[['upstream', 'resp_msg_id', 'reviewed-by']]\
                    .groupby('upstream')\
                    .count()['reviewed-by']

patches = pd.concat([patches_reviews, patches_emails], axis=1, sort=False)
patches.head()

In [ ]:
patches[['resp_msg_id', 'reviewed-by']].corr()

In [ ]:
del patches_emails 
del patches_reviews

Here I investigate whether there is a correlation between the number of emails involved in the discussion for each upstream commit and the number of reviews (that is, again, the number of emails including reviewed by tags) made for during the evaluation of that commit

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(patches['resp_msg_id'],
                                                               patches['reviewed-by'])

plt.plot(patches['resp_msg_id'],patches['reviewed-by'], 'o', label='original data')
# plt.plot(patches['resp_msg_id'], 
#          intercept + slope*patches['resp_msg_id'], 
#          'r', 
#          label='fitted line')

plt.xlabel('Number of emails')
plt.ylabel('Number of reviews')
plt.title('Counts per patch')

fig1 = plt.gcf()
fig1.savefig('review_to_reponse_peresponse_to_email_patch.png', dpi=1000, bbox_inches = "tight")

plt.legend()
plt.show()

In the graph above, each dot represents an upstream commit. Number of emails involved in the discussion about each commit and the number of reviews are shown in the horizontal and vertical axes, respectively. 

In [ ]:
del patches

#### Compare number of Reviewed-by vs. email responses per author 
(Do Reviewed-by tags and number of emails generally correlate among authors?)

What is the Reviewed-by to email responses ratio per author? Is this distribution of ratio concentrated around one value (there is "one common ratio preference among developers"), equally distributed in the scale (everyone has a different preference of use), or clustered among the scale (there are different parties of preference).

In [ ]:
authors_emails = patch_responses_long\
                    .groupby('author')\
                    .nunique()['resp_msg_id']
authors_reviews = patch_responses_long\
                    .groupby('author')\
                    .count()['reviewed-by']

authors = pd.concat([authors_reviews, authors_emails], axis=1, sort=False)
authors.head()

In [ ]:
del authors_emails 
del authors_reviews

This ratio probably doesn't mean too much, but we do know that there's a significant corr between 'resp_msg_id', 'reviewed-by'. What to do with them?

#### Does many responses mean many reviews (per author)?

In [ ]:
authors[['resp_msg_id', 'reviewed-by']].corr()

In [ ]:
slope, intercept, r_value, p_value, std_err = stats.linregress(authors['resp_msg_id'],
                                                               authors['reviewed-by'])

plt.plot(authors['resp_msg_id'], authors['reviewed-by'], 'o')
#plt.plot(authors['resp_msg_id'], 
#             intercept + slope*authors['resp_msg_id'], 
#             'r', 
#             label='fitted line')
#plt.legend()

plt.xlabel('Number of emails')
plt.ylabel('Number of reviews')
plt.title('Counts per author')

fig1 = plt.gcf()
fig1.savefig('review_to_response_per_author.png', dpi=1000, bbox_inches = "tight")

plt.show()

In [ ]:
del authors

In the graph above, each dot represents a developer. Number of emails involved in the discussion about each developer's patches and the number of reviews are shown in the horizontal and vertical axes, respectively.

#### Individuals

who is reviewing patches? Simply plain number of emails of overall reviewers in the whole kernel
Development Groups

In [ ]:
ax = patch_responses_long[['reviewed-by', 'resp_msg_id']]\
                                    .groupby('reviewed-by')\
                                    .nunique()['resp_msg_id']\
                                    .reset_index()\
                                    .sort_values('resp_msg_id', ascending=False)[:10]\
                                    .plot(x ='reviewed-by', title= 'Top reviewers', kind = 'barh')
ax.figure.savefig('top_reviewers.png', dpi=1000, bbox_inches = "tight")   

Above are the top 20 reviewers who has sent the largest number of reviews. For these counts, every email which has a Reviewed-by tag is considered a review.

#### Development Groups

Who reviews patches from whom? Relationship among developers
Can we identify groups based on who authors patches and who reviews from each other? 

In [ ]:
patch_responses_relations = patch_responses_long[['author', 'reviewed-by', 'resp_msg_id']]\
                                    .groupby(['author', 'reviewed-by'])\
                                    .nunique()['resp_msg_id']\
                                    .reset_index()
patch_responses_relations.head()

In [ ]:
patch_responses_relations.to_csv('patch_responses_relations.csv')

In [ ]:
ax = patch_responses_relations.plot(kind = 'hist', 
                              title = 'Distribution of message counts from each reviewer author pair', 
                              bins = 50)
ax.figure.savefig('author_reviewer_reviewcount_dist.png', dpi=1000, bbox_inches = "tight")

### Among a group of authors and reviewers (within a cluster), how is authoring and reviewing distributed among the group of developers?

 To investigate relations between persons both from the groups of reviewers and developers, an undirected graph is created. Each node in the graph repsresent a person. Edges represent email exchange between two persons. Edges are weighted according to number of emails.

This graph demonstrates all of the reviews made in development process together with the authors of the patches that have been reviewed.

 The Louvain method for community detection is used to divide the graph into groups. Communities are represented with node colors in the visualization.

 For the sake of simplicity and readibility, only three levels of edge weights are shown in the visualization. Darker edges represent more email exchange.

In [ ]:
# Create undirected graph with weighted edges

import networkx as nx

G=nx.Graph()

G.add_nodes_from(patch_responses_relations['reviewed-by'])
G.add_nodes_from(patch_responses_relations['author'])

G.add_weighted_edges_from(patch_responses_relations.values.tolist())

Determine partitioning

In [ ]:
from community import community_louvain
partition = community_louvain.best_partition(G)

Create graph layout, refer to: https://stackoverflow.com/questions/21978487/improving-python-networkx-graph-layout

In [ ]:
# pos = nx.kamada_kawai_layout(G)
pos = nx.nx_agraph.graphviz_layout(G)

Draw graph

In [ ]:
# import matplotlib.cm as cm
import itertools
import numpy as np

colors = itertools.cycle(['b', 'g', 'r', 'c', 'm', 'y', 'k'])

# size = float(len(set(partition.values())))
# Draw nodes, colored according to partitioning
count = 0.
for com in set(partition.values()) :
    count = count + 1.
    list_nodes = [nodes for nodes in partition.keys()
                                if partition[nodes] == com]
    nx.draw_networkx_nodes(G, pos, list_nodes, node_size = 3,
                        node_color = np.array([next(colors)]))
    edges, weights = zip(*nx.get_edge_attributes(G,'weight').items())

# To divide the edge weights into bins proportional to max
max_weight = max(weights)
# Divide edge weights into 3 categories
elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] >= 0.66*max_weight]
emedium = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] >= 0.1*max_weight and  d['weight'] < 0.66*max_weight]
esmall = [(u, v) for (u, v, d) in G.edges(data=True) if d['weight'] < 0.1*max_weight]

# edges, 3 categories
nx.draw_networkx_edges(G, pos, edgelist=elarge, edge_color=str(1))
nx.draw_networkx_edges(G, pos, edgelist=emedium, edge_color=str(0.66))
nx.draw_networkx_edges(G, pos, edgelist=esmall, edge_color=str(0.33))

# edges, the darker, the greater weight
# nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color=weights, edge_cmap=cm.Greys)

# If we want labels for each person:
# nx.draw_networkx_labels(G, pos, font_size = 1)

fig1 = plt.gcf()
plt.show()
plt.draw()
fig1.savefig('plot_mails.png', dpi=1000, bbox_inches = "tight")

In [ ]:
del patch_responses_relations

### Maintainers, reviewers and patch authors

In [ ]:
from pypasta.LinuxMaintainers import LinuxMaintainers
from pypasta.Util import file_to_string
from pypasta.Repository.Repository import Repository

def pretty_name(maintainer):
    try:
        if isinstance(maintainer, str):
            return None
        else:
            return '"' + maintainer[0][:-1] + '"' + ' <'+maintainer[1]+'>'
    except:
        # Return None when it is a mailing list or it is empty 
        return None

def flatten(nested_list):
    return [item for sublist in nested_list for item in sublist]

f_maintainers = file_to_string('./resources/linux/repo/MAINTAINERS')
all_maintainers = LinuxMaintainers(f_maintainers)
repo = Repository('./resources/linux/repo')

def get_maintainer(upstream):
    try:
        maintainers = []
        filenames = repo.get_commit(upstream).diff.affected
        subsystems = all_maintainers.get_subsystems_by_files(filenames)            
        for subsystem in subsystems:
            maintainers += all_maintainers.get_maintainers(subsystem)
        maintainers = {pretty_name(maintainer) for maintainer in flatten(maintainers)}
        return list(maintainers)
    except:
        return None

In [ ]:
# patch_responses_long = pd.read_csv('df_with_tags.csv')

In [ ]:
maintainer = patch_responses_long[['upstream', 'author', 'reviewed-by', 'resp_msg_id']]

In [ ]:
del patch_responses_long

In [ ]:
maintainer['maintainer'] = maintainer['upstream'].apply(get_maintainer)

In [ ]:
maintainer.set_index(['upstream', 'author', 'reviewed-by', 'resp_msg_id'], inplace=True)

In [ ]:
maintainer = pd.melt(maintainer['maintainer'].apply(pd.Series).reset_index(), 
            id_vars=['upstream', 'author', 'reviewed-by', 'resp_msg_id'],
            value_name='maintainer').sort_index()
maintainer.drop('variable', axis=1, inplace=True)
maintainer = maintainer.dropna(how='all')

It takes some time to get maintainers from hashes, even for smaller samples. I save my previous results and start from there for analysis.

In [ ]:
maintainer.to_csv('maintainer.csv')
# import pandas as pd
# maintainer = pd.read_csv('maintainer.csv', 
#                          usecols=['upstream', 'author', 'reviewed-by', 'resp_msg_id', 'maintainer'],
#                          dtype = {'upstream':str, 'author':str, 'reviewed-by':str, 
#                                   'resp_msg_id':str, 'maintainer':str})

In [ ]:
maintainer.head()

In [ ]:
counts = maintainer.groupby('.resp_msg_id').nunique()
counts.head()

In [ ]:
maintainer[maintainer['maintainer'] == maintainer['reviewed-by']]

In [ ]:
counts.plot.scatter('reviewed-by', 'maintainer')

Messages where one of maintainers is the patch author

In [ ]:
maintainer[maintainer['maintainer'] == maintainer['author']]

Number of emails in each maintainer's area

In [ ]:
maintainer.groupby(['maintainer']).nunique()['resp_msg_id'].sort_values(ascending=False)

Number of emails from each reviewer, for comparison

In [ ]:
maintainer.groupby(['reviewed-by']).nunique()['resp_msg_id'].sort_values(ascending=False)

Messages where one of maintainers is the reviewer

In [ ]:
maintainer[maintainer['maintainer'] == maintainer['reviewed-by']]

The maintainers are detected for a very limited subset of the dataset, as we only ĺook at upstreams that ended up in the repo, this could possibly explain not getting any results above.